<a href="https://colab.research.google.com/github/FREDY129053/Final_theory_work/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Обучение модель для предсказания акций IBM, Yandex, MIME

## Импорт библиотек

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

## Работа с датасетом, полученным из БД

In [58]:
df = pd.read_csv('/content/ibm_data.csv')

In [59]:
df['Date_point'] = pd.to_datetime(df['Date_point'])

In [60]:
df.head(5)

,Date_point,Name,Open,High,Low,Close,Volume
0,2018-12-31,International Business Machines Corp,113.33,114.35,112.4201,113.67,4982755.0
1,2018-12-28,International Business Machines Corp,114.22,114.80,112.5000,113.03,5117533.0
2,2018-12-27,International Business Machines Corp,109.99,113.78,109.4700,113.78,6045590.0
3,2018-12-26,International Business Machines Corp,108.00,111.39,105.9400,111.39,6645794.0
4,2018-12-24,International Business Machines Corp,109.90,111.00,107.5000,107.57,3821376.0


In [61]:
df.tail(5)

,Date_point,Name,Open,High,Low,Close,Volume
4817,1999-11-05,International Business Machines Corp,92.75,92.94,90.19,90.25,13737600.0
4818,1999-11-04,International Business Machines Corp,94.44,94.44,90.00,91.56,16697600.0
4819,1999-11-03,International Business Machines Corp,95.87,95.94,93.50,94.37,10369100.0
4820,1999-11-02,International Business Machines Corp,96.75,96.81,93.69,94.81,11105400.0
4821,1999-11-01,International Business Machines Corp,98.50,98.81,96.37,96.75,9551800.0


In [62]:
copy_df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
copy_df.shape

(4822, 5)

In [63]:
np_ds_scaled = np.reshape(copy_df, (copy_df.shape[0], copy_df.shape[1]))
np_ds_scaled = np_ds_scaled[::-1]
np_ds_scaled

array([[9.850000e+01, 9.881000e+01, 9.637000e+01, 9.675000e+01,
        9.551800e+06],
       [9.675000e+01, 9.681000e+01, 9.369000e+01, 9.481000e+01,
        1.110540e+07],
       [9.587000e+01, 9.594000e+01, 9.350000e+01, 9.437000e+01,
        1.036910e+07],
       ...,
       [1.099900e+02, 1.137800e+02, 1.094700e+02, 1.137800e+02,
        6.045590e+06],
       [1.142200e+02, 1.148000e+02, 1.125000e+02, 1.130300e+02,
        5.117533e+06],
       [1.133300e+02, 1.143500e+02, 1.124201e+02, 1.136700e+02,
        4.982755e+06]])

## Обучение модели

### Подготовка к обучению

In [64]:
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps_in
      out_end_ix = end_ix + n_steps_out
      # check if we are beyond the sequence
      if out_end_ix > len(sequence):
        break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
      X.append(seq_x)
      tmp = [i[-2] for i in seq_y]
      y.append(tmp)
  return np.array(X), np.array(y)

In [65]:
n_steps_in, n_steps_out = 40, 90
x, y = split_sequence(np_ds_scaled, n_steps_in, n_steps_out)

In [66]:
x = x.reshape((x.shape[0], x.shape[1], 5))

In [67]:
# train_size = int(len(ds_scaled)*0.70)
# test_size = len(ds_scaled) - train_size

splitlimit = int(len(x)*0.8)
x_train, x_test = x[:splitlimit], x[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]

In [68]:
from sklearn.preprocessing import MinMaxScaler

normalizer1 = MinMaxScaler(feature_range=(0,1))
normalizer2 = MinMaxScaler(feature_range=(0,1))

x_train_tmp = x_train.reshape(-1, x_train.shape[-1])
x_test_tmp = x_test.reshape(-1, x_test.shape[-1])

x_train_scaled = normalizer1.fit_transform(x_train_tmp)
x_test_scaled = normalizer2.fit_transform(x_test_tmp)

x_train = x_train_scaled.reshape(x_train.shape)
x_test = x_test_scaled.reshape(x_test.shape)

### Обучение модели

#### Модель для IBM

In [72]:
model = Sequential()

model.add(LSTM(250, activation='relu', return_sequences=True, input_shape=(n_steps_in, 5)))
model.add(Dropout(0.2))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=n_steps_out, activation='linear'))

model.compile(optimizer='adam', loss='mse')

history = model.fit(x_train, y_train, batch_size=15, epochs=60, shuffle=True, validation_split = 0.2)

Epoch 1/60
201/201 [==============================] - 34s 145ms/step - loss: 4160893184.0000 - val_loss: 56909.0078
Epoch 2/60
201/201 [==============================] - 29s 147ms/step - loss: 2612.6907 - val_loss: 384.1785
Epoch 3/60
201/201 [==============================] - 29s 146ms/step - loss: 760.0545 - val_loss: 494.2050
Epoch 4/60
201/201 [==============================] - 29s 143ms/step - loss: 620.5889 - val_loss: 309.6855
Epoch 5/60
201/201 [==============================] - 29s 145ms/step - loss: 631.0406 - val_loss: 657.6804
Epoch 6/60
201/201 [==============================] - 29s 145ms/step - loss: 614.8704 - val_loss: 1051.4255
Epoch 7/60
201/201 [==============================] - 29s 144ms/step - loss: 677.5818 - val_loss: 208.1014
Epoch 8/60
201/201 [==============================] - 29s 146ms/step - loss: 590.1005 - val_loss: 385.7183
Epoch 9/60
201/201 [==============================] - 29s 146ms/step - loss: 556.2628 - val_loss: 218.1200
Epoch 10/60
201/201 [=====

## Результаты

### Результаты для IBM

In [73]:
predict = model.predict(x_test)
predict

30/30 [==============================] - 3s 78ms/step


array([[187.13466 , 186.17651 , 189.35567 , ..., 192.22723 , 193.06573 ,
        189.91228 ],
       [185.01985 , 184.22049 , 187.42786 , ..., 190.13014 , 191.11105 ,
        187.9956  ],
       [182.09933 , 181.4907  , 184.78307 , ..., 187.23528 , 188.43167 ,
        185.33472 ],
       ...,
       [103.955185, 103.800415, 103.46153 , ..., 105.61159 , 105.86517 ,
        106.51098 ],
       [104.22279 , 104.04708 , 103.723755, ..., 105.88839 , 106.12563 ,
        106.7568  ],
       [104.43361 , 104.23998 , 103.89858 , ..., 106.0855  , 106.29806 ,
        106.93169 ]], dtype=float32)

In [74]:
y_test

array([[161.54, 162.67, 164.52, ..., 161.85, 162.34, 162.86],
       [162.67, 164.52, 164.05, ..., 162.34, 162.86, 162.38],
       [164.52, 164.05, 163.27, ..., 162.86, 162.38, 162.3 ],
       ...,
       [146.51, 145.97, 145.24, ..., 107.57, 111.39, 113.78],
       [145.97, 145.24, 145.37, ..., 111.39, 113.78, 113.03],
       [145.24, 145.37, 146.04, ..., 113.78, 113.03, 113.67]])

In [75]:
model.save('/content/LSTM_IBM_NICE.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Результаты для Yandex